# Setup

In [2]:
# Import packages

import tensorflow as tf
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import RobustScaler
import warnings
import logging

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.6.4


In [3]:
# Random seed for reproducibility

seed = 69

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
# Remove warnings

import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Data Loading and Preprocessing

In [5]:
# Loading x

x = np.load("/kaggle/input/training-dataset-homework-2/x_train.npy")
print(x.shape)

(2429, 36, 6)


In [6]:
# Loading y

y = np.load("/kaggle/input/training-dataset-homework-2/y_train.npy")
print(y.shape)

(2429,)


In [7]:
# Define useful constants

OUTPUT_SHAPE = 12
FEATURE_NUMBER = 6
TIME_INTERVAL = 36

In [8]:
# Data splitting function into training and validation sets

def split_data(data, labels, train_percentage):
    
    y_count = np.zeros(OUTPUT_SHAPE,dtype=int)
    x_train = np.empty(shape=(0,36,6))
    x_val = np.empty(shape=(0,36,6))
    y_train = np.empty(shape=(0), dtype=int)
    y_val = np.empty(shape=(0), dtype=int)
    
    for l in labels:
        y_count[l] += 1
    
    for l in range(OUTPUT_SHAPE):
        n_train = (int)(y_count[l]*train_percentage)
        x_train = np.append(x_train, data[labels == l][:n_train, :, :], axis=0)
        y_train = np.append(y_train, labels[labels == l][:n_train])
        x_val = np.append(x_val, data[labels == l][n_train:, :, :], axis=0)
        y_val = np.append(y_val, labels[labels == l][n_train:])
        
    return x_train, x_val, y_train, y_val

In [9]:
# Data splitting into training set and validation set

x_train, x_val, y_train, y_val = split_data(x, y, 0.85)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(2059, 36, 6) (2059,) (370, 36, 6) (370,)


In [10]:
# Normalization function to normalize the data

def normalization_easier(data, val_data):
    x_train_pre = data.copy()
    x_val_pre = val_data.copy()
    scaler = RobustScaler()
    
    x_train_pre = scaler.fit_transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)      
    x_val_pre = scaler.transform(val_data.reshape(-1, val_data.shape[-1])).reshape(val_data.shape)
    return x_train_pre, x_val_pre

In [11]:
# Normalize the data

a, b = normalization_easier(x_train,x_val)
a.shape

(2059, 36, 6)

In [31]:
# Define a function to save into a file the useful parameters to perform normalization

from pickle import dump
def get_parameters_normalization(data, val_data):
    x_train_pre = data.copy()
    x_val_pre = val_data.copy()
    scaler = RobustScaler()
    scaler = scaler.fit(data.reshape(-1, data.shape[-1]))
    dump(scaler, open('scaler_15_12_second_model.pkl', 'wb'))

In [32]:
# Call the previous function to generate the pickle file, to be used in submission (test) phase

get_parameters_normalization(x_train, x_val)

# Model

In [12]:
# Define the input shape, the batch size and the number of epochs

input_shape = a.shape[1:]
batch_size = 32
epochs = 1000

In [25]:
def build_1DCNN_classifier(input_shape, classes):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    cnn = tfkl.Conv1D(filters=128,kernel_size=3,padding='same',activation='relu')(input_layer)
    d1 = tfkl.Dropout(.2, seed=seed)(cnn)
    cnn = tfkl.Conv1D(filters=128,kernel_size=3,padding='same',activation='relu')(d1)
    d2 = tfkl.Dropout(.2, seed=seed)(cnn)
    cnn = tfkl.Conv1D(filters=128,kernel_size=3,padding='same',activation='relu')(d2)
    d3 = tfkl.Dropout(.2, seed=seed)(cnn)
    gap = tfkl.GlobalAveragePooling1D()(d3)
    dropout = tfkl.Dropout(.5, seed=seed)(gap)

    # Classifier
    classifier = tfkl.Dense(512, activation='relu')(dropout)
    dropout2 = tfkl.Dropout(.5, seed=seed)(classifier)
    classifier2 = tfkl.Dense(64, activation='relu')(dropout2)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier2)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [26]:
# Get the summary of the built model

model = build_1DCNN_classifier(input_shape, OUTPUT_SHAPE)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 36, 6)]           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 36, 128)           2432      
_________________________________________________________________
dropout_15 (Dropout)         (None, 36, 128)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 36, 128)           49280     
_________________________________________________________________
dropout_16 (Dropout)         (None, 36, 128)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 36, 128)           49280     
_________________________________________________________________
dropout_17 (Dropout)         (None, 36, 128)           0     

In [27]:
# Fit the model

history = model.fit(
    x = a,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data=(b, y_val),
    class_weight = {0: 1,
                    1: 1,
                    2: 1,
                    3: 1,
                    4: 3,
                    5: 1,
                    6: 1,
                    7: 2,
                    8: 1,
                    9: 1,
                    10: 1,
                    11: 2},
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=10, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/1000
65/65 [==============================] - 1s 6ms/step - loss: 3.1003 - accuracy: 0.2807 - val_loss: 2.3169 - val_accuracy: 0.3459
Epoch 2/1000
65/65 [==============================] - 0s 4ms/step - loss: 2.4682 - accuracy: 0.3570 - val_loss: 2.0473 - val_accuracy: 0.3676
Epoch 3/1000
65/65 [==============================] - 1s 8ms/step - loss: 2.2838 - accuracy: 0.3667 - val_loss: 1.8819 - val_accuracy: 0.4027
Epoch 4/1000
65/65 [==============================] - 0s 5ms/step - loss: 2.1891 - accuracy: 0.3798 - val_loss: 1.9010 - val_accuracy: 0.4135
Epoch 5/1000
65/65 [==============================] - 0s 4ms/step - loss: 2.1722 - accuracy: 0.3978 - val_loss: 1.8287 - val_accuracy: 0.4297
Epoch 6/1000
65/65 [==============================] - 0s 4ms/step - loss: 2.1448 - accuracy: 0.4235 - val_loss: 1.7621 - val_accuracy: 0.4514
Epoch 7/1000
65/65 [==============================] - 0s 4ms/step - loss: 2.0656 - accuracy: 0.4264 - val_loss: 1.6526 - val_accuracy: 0.4703
Epoch 

In [ ]:
# Code to generate the plots of training/validation loss and training/validation accuracy of the model

best_epoch = np.argmax(history['val_accuracy'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.title('Categorical Crossentropy')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['accuracy'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.title('Accuracy')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [28]:
# Save the model

model.save('15_12_second_model')

In [29]:
# Create a zip of the saved model

import shutil
shutil.make_archive("15_12_second_model", 'zip', "/kaggle/working/15_12_second_model")

'/kaggle/working/15_12_second_model.zip'